
# 03_multimodal_llm_decoder_training.ipynb

**Notebook 3 – Aligned Model + “Normal” Decoder (Standard LLM)**

This notebook connects the **Phase‑1 aligned model** (vision–text or tri‑modal Perceiver alignment) to a standard decoder‑only LLM (e.g., Qwen2.5‑1.5B) and trains a **vision‑conditioned captioning / multimodal generation head**.

Key features:

- Load **pre‑trained alignment checkpoint** (Phase‑1)
- Build a **MultimodalLLM** that uses the aligned vision embedding as a **soft‑prompt prefix** for a frozen LLM
- Use **Valor tri‑modal parquet** (or compatible) for training (image+text; extendable to audio)
- **Multi‑GPU training** via `DataParallel` (2 GPUs supported)
- **Weights & Biases** logging for:
  - Training & validation loss
  - Learning rate
  - Example generations (tables with GT vs predicted captions)
- Clean experiment directory structure & checkpoint saving

> 🔧 You are expected to adjust a few paths (e.g., `VALOR_PARQUET_PATH`, `IMAGE_ROOT_DIR`, `ALIGN_CKPT_PATH`) to match your environment.


## 1. Imports and Environment Setup

In [1]:

import os
import math
import time
from pathlib import Path
from dataclasses import dataclass, asdict
from typing import Dict, Any, Optional, List

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

import pandas as pd
from PIL import Image
from tqdm.auto import tqdm

import wandb

# Local modules
from imports.core import AlignmentConfig, VisionTextAligner, set_seed, count_parameters  # phase‑1 aligner
from imports.llm_integration import (
    LLMConfig,
    MultimodalLLM,
    create_caption_labels,
    train_multimodal_step,
)
from imports.train import load_checkpoint  # for loading phase‑1 adapter weights

torch.backends.cudnn.benchmark = True


/home/hice1/vchopra37/scratch/projects/edge_glass/edge_glass_env/lib/python3.12/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 2. Experiment Configuration

In [ ]:

@dataclass
class Phase3Config:
    # ----- Paths -----
    project_name: str = "edgeglass_phase3_normal_llm"
    run_name: str = "valor_tri_modal_qwen_prefix"
    
    # Base experiment directory (all artifacts under this)
    base_dir: str = "./checkpoints/phase1"
    
    # Phase‑1 alignment checkpoint (vision‑text or tri‑modal)
    align_checkpoint_path: str = "./checkpoints/phase1/best.pt"
    
    # Valor / tri‑modal parquet path (image + text; optionally audio)
    valor_parquet_path: str = "./data/alignment_subsets/valor32k_train_shards/*.parquet"
    image_root_dir: str = "./data/alignment_subsets/valor32k_train_shards/images"  # root dir to prepend to relative image paths
    
    # Column names in Valor parquet
    image_path_col: str = "image_path"    # path or relative path to image
    caption_col: str = "caption"          # text caption; adjust if "text" etc.
    
    # Train/val split
    val_ratio: float = 0.05
    max_rows: Optional[int] = None  # set to e.g. 50_000 to cap rows
    
    # ----- LLM & prefix -----
    llm_model_name: str = "Qwen/Qwen2.5-1.5B-Instruct"
    freeze_llm: bool = True
    num_prefix_tokens: int = 8
    
    # ----- Training -----
    num_epochs: int = 3
    batch_size_per_gpu: int = 4
    learning_rate: float = 5e-5
    weight_decay: float = 0.01
    max_grad_norm: float = 1.0
    warmup_ratio: float = 0.1
    max_caption_len: int = 64
    
    # Logging
    log_every: int = 50
    eval_every: int = 500  # steps
    num_eval_samples: int = 8  # qualitative generations
    
    # Device / precision
    seed: int = 42
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    dtype: str = "bfloat16" if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else "float16"

cfg = Phase3Config()

# Create directories
base_dir = Path(cfg.base_dir)
ckpt_dir = base_dir / "checkpoints"
log_dir = base_dir / "logs"
samples_dir = base_dir / "samples"

for d in [ckpt_dir, log_dir, samples_dir]:
    d.mkdir(parents=True, exist_ok=True)

print("Base dir:", base_dir)
print("Checkpoint dir:", ckpt_dir)
print("Valor parquet:", cfg.valor_parquet_path)


## 3. Utility Functions

In [ ]:

def get_device() -> torch.device:
    return torch.device(cfg.device)

def get_dtype() -> torch.dtype:
    if cfg.dtype == "bfloat16":
        return torch.bfloat16
    if cfg.dtype == "float16":
        return torch.float16
    return torch.float32

def setup_seed():
    set_seed(cfg.seed)
    torch.cuda.empty_cache()
    print(f"Seed set to {cfg.seed}")

setup_seed()

device = get_device()
dtype = get_dtype()
print(f"Using device: {device} | dtype: {dtype}")
print(f"GPUs available: {torch.cuda.device_count()}")
if torch.cuda.device_count() > 0:
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")


## 4. Load Phase‑1 Aligner Checkpoint

In [ ]:

# NOTE: AlignmentConfig here should match what you used in Phase‑1.
align_cfg = AlignmentConfig()
align_cfg.device = device
align_cfg.dtype = dtype

aligner = VisionTextAligner(align_cfg).to(device)

print("Loading Phase‑1 alignment checkpoint from:", cfg.align_checkpoint_path)
_ = load_checkpoint(
    model=aligner,
    checkpoint_path=cfg.align_checkpoint_path,
    load_optimizer=False,
    optimizer=None,
)

# Freeze aligner completely for Phase‑3 (decoder training)
for p in aligner.parameters():
    p.requires_grad = False

print("Aligner parameter counts:", count_parameters(aligner))


## 5. Build Multimodal LLM (Aligned Model + Normal Decoder)

In [ ]:

llm_cfg = LLMConfig(
    model_name=cfg.llm_model_name,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9,
    num_prefix_tokens=cfg.num_prefix_tokens,
    freeze_llm=cfg.freeze_llm,
)

multimodal_model = MultimodalLLM(
    aligner=aligner,
    llm_config=llm_cfg,
)

multimodal_model.to(device)
multimodal_model = multimodal_model.to(dtype=dtype)

# Multi‑GPU (DataParallel)
num_gpus = torch.cuda.device_count()
if num_gpus > 1:
    print(f"Wrapping MultimodalLLM with DataParallel over {num_gpus} GPUs")
    multimodal_model = nn.DataParallel(multimodal_model)

# Convenience handle to the underlying module for attributes (tokenizer, etc.)
mm_module = multimodal_model.module if isinstance(multimodal_model, nn.DataParallel) else multimodal_model

print("Multimodal model ready.")
print("Trainable params (projector only in Phase‑2a):", sum(p.numel() for p in mm_module.get_trainable_params()))


## 6. Valor Tri‑Modal Dataset (Image + Text)

In [ ]:

class ValorImageTextDataset(Dataset):
    """
    Simple dataset for Valor‑style parquet with at least:
      - image path column (absolute or relative to `image_root_dir`)
      - caption/text column
    
    You can later extend this to include audio features, pre‑extracted encodings, etc.
    """
    def __init__(
        self,
        df: pd.DataFrame,
        image_root_dir: Path,
        image_path_col: str,
        caption_col: str,
        tokenizer,
        max_length: int = 64,
    ):
        self.df = df.reset_index(drop=True)
        self.image_root = Path(image_root_dir)
        self.image_path_col = image_path_col
        self.caption_col = caption_col
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Pre‑tokenize captions for efficiency & to make DataLoader workers simple
        self._tokenized = []
        for _, row in tqdm(self.df.iterrows(), total=len(self.df), desc="Tokenizing captions"):
            caption = str(row[self.caption_col])
            tokens = self.tokenizer(
                caption,
                padding="max_length",
                truncation=True,
                max_length=self.max_length,
                return_tensors="pt",
            )
            input_ids = tokens["input_ids"][0]
            attention_mask = tokens["attention_mask"][0]
            labels = input_ids.clone()
            labels[attention_mask == 0] = -100
            self._tokenized.append(
                {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                    "labels": labels,
                    "raw_caption": caption,
                }
            )

    def __len__(self) -> int:
        return len(self.df)

    def _load_image(self, path: str) -> Image.Image:
        p = Path(path)
        if not p.is_absolute():
            p = self.image_root / p
        img = Image.open(p).convert("RGB")
        return img

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        row = self.df.iloc[idx]
        path = row[self.image_path_col]
        img = self._load_image(path)
        toks = self._tokenized[idx]
        return {
            "image": img,
            "input_ids": toks["input_ids"],
            "attention_mask": toks["attention_mask"],
            "labels": toks["labels"],
            "caption": toks["raw_caption"],
            "image_path": str(path),
        }


def valor_collate_fn(batch: List[Dict[str, Any]]) -> Dict[str, Any]:
    images = [b["image"] for b in batch]
    input_ids = torch.stack([b["input_ids"] for b in batch], dim=0)
    attention_mask = torch.stack([b["attention_mask"] for b in batch], dim=0)
    labels = torch.stack([b["labels"] for b in batch], dim=0)
    captions = [b["caption"] for b in batch]
    image_paths = [b["image_path"] for b in batch]

    return {
        "images": images,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
        "captions": captions,
        "image_paths": image_paths,
    }


## 7. Load Valor Parquet and Build DataLoaders

In [ ]:

print("Loading Valor parquet from:", cfg.valor_parquet_path)
valor_df = pd.read_parquet(cfg.valor_parquet_path)
print("Raw Valor rows:", len(valor_df))

if cfg.max_rows is not None:
    valor_df = valor_df.head(cfg.max_rows)
    print("Capped Valor rows to:", len(valor_df))

# Basic sanity check
print("Columns:", list(valor_df.columns)[:20])
assert cfg.image_path_col in valor_df.columns, f"Missing image column: {cfg.image_path_col}"
assert cfg.caption_col in valor_df.columns, f"Missing caption column: {cfg.caption_col}"

# Train/val split
n_total = len(valor_df)
n_val = max(1, int(n_total * cfg.val_ratio))
n_train = n_total - n_val

train_df = valor_df.iloc[:n_train].reset_index(drop=True)
val_df = valor_df.iloc[n_train:].reset_index(drop=True)

print(f"Train rows: {len(train_df)} | Val rows: {len(val_df)}")


tokenizer = mm_module.llm.tokenizer

train_dataset = ValorImageTextDataset(
    df=train_df,
    image_root_dir=Path(cfg.image_root_dir),
    image_path_col=cfg.image_path_col,
    caption_col=cfg.caption_col,
    tokenizer=tokenizer,
    max_length=cfg.max_caption_len,
)

val_dataset = ValorImageTextDataset(
    df=val_df,
    image_root_dir=Path(cfg.image_root_dir),
    image_path_col=cfg.image_path_col,
    caption_col=cfg.caption_col,
    tokenizer=tokenizer,
    max_length=cfg.max_caption_len,
)

global_batch_size = cfg.batch_size_per_gpu * max(1, torch.cuda.device_count())

train_loader = DataLoader(
    train_dataset,
    batch_size=global_batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=torch.cuda.is_available(),
    collate_fn=valor_collate_fn,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=global_batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=torch.cuda.is_available(),
    collate_fn=valor_collate_fn,
)

print(f"Batches per epoch: train={len(train_loader)}, val={len(val_loader)}"))


## 8. Optimizer and Scheduler

In [ ]:

# Only projector params are trainable in Phase‑2a
trainable_params = list(mm_module.get_trainable_params())
print("Trainable parameter count:", sum(p.numel() for p in trainable_params))

optimizer = AdamW(
    trainable_params,
    lr=cfg.learning_rate,
    weight_decay=cfg.weight_decay,
)

num_training_steps = len(train_loader) * cfg.num_epochs
warmup_steps = int(num_training_steps * cfg.warmup_ratio)
print(f"Total training steps: {num_training_steps} | Warmup steps: {warmup_steps}")

scheduler = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=max(10, num_training_steps - warmup_steps),
    T_mult=1,
)


## 9. Weights & Biases Setup

In [ ]:

wandb_run = wandb.init(
    project=cfg.project_name,
    name=cfg.run_name,
    config=asdict(cfg),
)

wandb.watch(mm_module.projector, log="all", log_freq=100)


## 10. Training & Evaluation Loop

In [ ]:

def evaluate_epoch(step: int) -> Dict[str, float]:
    mm_module.eval()
    total_loss = 0.0
    n_batches = 0

    all_samples = []  # for qualitative logging

    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(val_loader, desc=f"Eval @ step {step}", leave=False)):
            images = batch["images"]
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = mm_module(
                images=images,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
            )
            loss = outputs["loss"]
            total_loss += loss.item()
            n_batches += 1

            # Collect a few qualitative generations
            if len(all_samples) < cfg.num_eval_samples:
                for img, cap in zip(images, batch["captions"]):
                    if len(all_samples) >= cfg.num_eval_samples:
                        break
                    try:
                        gen_text = mm_module.generate(
                            images=img,
                            prompt="Describe this scene in one or two sentences:",
                            max_new_tokens=64,
                            temperature=0.7,
                        )
                    except Exception as e:
                        gen_text = f"[GENERATION_ERROR: {e}]"
                    all_samples.append(
                        {
                            "image": wandb.Image(img),
                            "gt_caption": cap,
                            "gen_caption": gen_text,
                        }
                    )

    avg_loss = total_loss / max(1, n_batches)
    metrics = {"val_loss": avg_loss}

    # Log qualitative samples as W&B Table
    if all_samples:
        table = wandb.Table(columns=["image", "gt_caption", "gen_caption"])
        for s in all_samples:
            table.add_data(s["image"], s["gt_caption"], s["gen_caption"])
        wandb.log({"eval_samples": table, "global_step": step})

    return metrics


global_step = 0
best_val_loss = math.inf

mm_module.train()

for epoch in range(cfg.num_epochs):
    print(f"\n===== Epoch {epoch+1}/{cfg.num_epochs} =====")
    epoch_loss = 0.0
    n_batches = 0

    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Train Epoch {epoch+1}")):
        multimodal_model.train()  # handles DataParallel as well

        images = batch["images"]
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad(set_to_none=True)

        outputs = mm_module(
            images=images,
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )

        loss = outputs["loss"]
        loss.backward()

        torch.nn.utils.clip_grad_norm_(trainable_params, cfg.max_grad_norm)
        optimizer.step()
        scheduler.step(global_step)

        step_loss = loss.item()
        epoch_loss += step_loss
        n_batches += 1
        global_step += 1

        if global_step % cfg.log_every == 0:
            lr = optimizer.param_groups[0]["lr"]
            avg_loss = epoch_loss / max(1, n_batches)
            log_data = {
                "train_loss": step_loss,
                "train_loss_avg": avg_loss,
                "lr": lr,
                "epoch": epoch + 1,
                "global_step": global_step,
            }
            wandb.log(log_data)

        if global_step % cfg.eval_every == 0:
            print(f"\n>>> Running evaluation at step {global_step}...")
            val_metrics = evaluate_epoch(global_step)
            wandb.log({**val_metrics, "global_step": global_step})
            val_loss = val_metrics["val_loss"]

            # Save best checkpoint by val_loss
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                ckpt_path = ckpt_dir / "best_phase3.pt"
                torch.save(
                    {
                        "step": global_step,
                        "epoch": epoch,
                        "model_state_dict": mm_module.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                        "scheduler_state_dict": scheduler.state_dict(),
                        "cfg": asdict(cfg),
                    },
                    ckpt_path,
                )
                print(f"[BEST] Saved new best checkpoint to {ckpt_path} | val_loss={val_loss:.4f}")

    avg_epoch_loss = epoch_loss / max(1, n_batches)
    print(f"Epoch {epoch+1} complete | avg_train_loss={avg_epoch_loss:.4f}")
    wandb.log({"epoch_train_loss": avg_epoch_loss, "epoch": epoch + 1, "global_step": global_step})

# Final evaluation
print("\n>>> Final evaluation after training...")
final_val_metrics = evaluate_epoch(global_step)
wandb.log({**final_val_metrics, "global_step": global_step})

# Save final checkpoint
final_ckpt_path = ckpt_dir / "final_phase3.pt"
torch.save(
    {
        "step": global_step,
        "epoch": cfg.num_epochs,
        "model_state_dict": mm_module.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "cfg": asdict(cfg),
    },
    final_ckpt_path,
)
print(f"Saved final checkpoint to {final_ckpt_path}")


## 11. Inference Helper (Manual Testing)

In [ ]:

def generate_for_single_row(row_idx: int = 0, prompt: str = "Describe this scene:") -> Dict[str, Any]:
    """Quick helper to inspect one Valor sample and generation."""
    mm_module.eval()
    row = val_df.iloc[row_idx]
    path = row[cfg.image_path_col]
    p = Path(path)
    if not p.is_absolute():
        p = Path(cfg.image_root_dir) / p
    img = Image.open(p).convert("RGB")

    gen_text = mm_module.generate(
        images=img,
        prompt=prompt,
        max_new_tokens=96,
        temperature=0.7,
    )
    return {
        "image": img,
        "gt_caption": str(row[cfg.caption_col]),
        "gen_caption": gen_text,
    }

# Example (won't display in headless run, but useful in notebook):
# sample = generate_for_single_row(0)
# display(sample["image"])
# print("GT:", sample["gt_caption"])
# print("GEN:", sample["gen_caption"])
